In [45]:
# List of Yelp API Keys
api_keys = [
    "apikeyabc1",
    "apikeyabc2",
    "apikeyabc3",
    "apikeyabc4"
]

In [46]:
import requests
import folium

# Your Yelp API key
api_key = 'apikeyabc1'
headers = {'Authorization': f'Bearer {api_key}'}
url = 'https://api.yelp.com/v3/businesses/search'

# Stadiums to find
stadiums = {
    "MetLife Stadium": "East Rutherford, New Jersey",
    "AT&T Stadium": "Arlington, Dallas",
    "Arrowhead Stadium": "Kansas City",
    "NRG Stadium": "Houston",
    "Mercedes-Benz Stadium": "Atlanta",
    "SoFi Stadium": "Inglewood, Los Angeles",
    "Lumen Field": "Seattle",
    "Lincoln Financial Field": "Philadelphia",
    "Levi’s Stadium": "Santa Clara, San Francisco Bay Area",
    "Gillette Stadium": "Foxborough, Boston",
    "Hard Rock Stadium": "Miami Gardens, Miami"
}

# Dictionary to hold the latitude and longitude
coordinates = {}

for stadium_name, location in stadiums.items():
    city = location.split(',')[0]  # Assuming the city is the first part of the location string
    params = {'term': stadium_name, 'location': location}
    try:
        response = requests.get(url, headers=headers, params=params)
        data = response.json()
        if data['businesses']:
            first_business = data['businesses'][0]  # Assuming the first result is the desired one
            lat = first_business['coordinates']['latitude']
            long = first_business['coordinates']['longitude']
            coordinates[city] = (lat, long)
        else:
            coordinates[city] = "Not found"
    except Exception as e:
        print(f"An error occurred while fetching data for {stadium_name} in {city}: {e}")
        coordinates[city] = "Error"

# Printing the coordinates in the desired format
for city, coord in coordinates.items():
    print(f"{city}: {coord}")

# Create a map centered around the first coordinate in the list, if available
if coordinates:
    first_coord = next(iter(coordinates.values()))
    if isinstance(first_coord, tuple):
        map_ = folium.Map(location=[first_coord[0], first_coord[1]], zoom_start=5)
        for city, coord in coordinates.items():
            if isinstance(coord, tuple):
                folium.Marker([coord[0], coord[1]], popup=city, icon=folium.Icon(color='blue')).add_to(map_)
        # Save the map as an HTML file
        map_.save("C:/Users/agutierrez/Downloads/stadiums_map.html")
        print("Map has been saved as stadiums_map.html. You can now open this file in your web browser.")
    else:
        print("No valid coordinates found to display on the map.")
else:
    print("No coordinates were fetched.")


East Rutherford: (40.8128192553795, -74.0741152688998)
Arlington: (32.747284, -97.0945)
Kansas City: (39.0489393746087, -94.48400255221)
Houston: (29.6850688988644, -95.4109811782837)
Atlanta: (33.7555059715783, -84.4009642033808)
Inglewood: (33.953514, -118.33904161398364)
Seattle: (47.595162, -122.33164)
Philadelphia: (39.9008144703153, -75.1675401693115)
Santa Clara: (37.403319481674686, -121.96937885562004)
Foxborough: (42.0909219738127, -71.2643583118916)
Miami Gardens: (25.9575044705781, -80.2389049530029)
Map has been saved as stadiums_map.html. You can now open this file in your web browser.


In [47]:
import requests
import pandas as pd

# Ensure the needed variables are initialized 'coordinates' and 'API key' by running previous cells
# Var syntax:
# coordinates = {'CityName': (latitude, longitude)}
# api_keys = ['abcdkey1','abcdkey2''abcdkey3']

categories = 'restaurants,food'  # This can be dynamically set elsewhere in your code

def fetch_data(latitude, longitude, offset=0):
    for api_key in api_keys:
        url = f"https://api.yelp.com/v3/businesses/search?latitude={latitude}&longitude={longitude}&radius=10000&categories={categories}&sort_by=distance&limit=50&offset={offset}"
        headers = {"Authorization": f"Bearer {api_key}"}
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429:
            print(f"API Key rate limited: {api_key}. Trying next key.")
            continue
        else:
            print(f"Failed to fetch data with API Key: {api_key}, Status Code: {response.status_code}")
            return None
    print("All API keys rate limited or invalid.")
    return None

for city, (latitude, longitude) in coordinates.items():
    unique_businesses = {}
    offset = 0
    print(f"Processing {city}...")
    while True:
        data = fetch_data(latitude, longitude, offset)
        if not data or 'businesses' not in data:
            print("No more data or failed to fetch data. Moving to next city.")
            break
        
        for business in data['businesses']:
            if business['id'] not in unique_businesses:
                categories = ', '.join([cat['title'] for cat in business['categories']])
                address = ', '.join(business['location']['display_address'])
                unique_businesses[business['id']] = {
                    'ID': business['id'],
                    'Name': business['name'],
                    'Image URL': business['image_url'],
                    'URL': business['url'],
                    'Is Closed': business['is_closed'],
                    'Review Count': business['review_count'],
                    'Categories': categories,
                    'Rating': business['rating'],
                    'Latitude': business['coordinates']['latitude'],
                    'Longitude': business['coordinates']['longitude'],
                    'Transactions': ', '.join(business['transactions']),
                    'Price': business.get('price', ''),
                    'Address': address,
                    'Phone': business['phone'],
                    'Display Phone': business['display_phone'],
                    'Distance': business['distance']
                }
                print(f"Added unique business: {business['name']} (ID: {business['id']})")
        offset += 50
        if len(data['businesses']) < 50:
            break  # Break if last batch of businesses is less than 50, indicating the last page of results
    
    # Convert dictionary to a pandas DataFrame
    df = pd.DataFrame.from_dict(unique_businesses, orient='index')
    
    # Update the file path to include city and categories dynamically
    excel_file_path = f'{city}_{categories}.xlsx'
    df.to_excel(excel_file_path, index=False)
    print(f"Data saved to '{excel_file_path}'")


Processing East Rutherford...
API Key rate limited: Oqc1bmwWIuMAyZjJIZbS6Ge6XZn2-Q8RAPjN7qtyd1I-7mVMHZjFnyfJezADk6kqFqRrHu01Z6ErjF6srJB48dtDChV7V0FRvX7BWBVkhX0Hpnnk6IsU0XjJpwfzZXYx. Trying next key.
Added unique business: Good To Go Organics (ID: Bk--H5IrIBWsANls0A6gFg)
Added unique business: Dunkin' (ID: 8nEj_7DnNZ4Pxkbd3FCj0g)
Added unique business: Dunkin' (ID: _52O96WhLxCR9Vt4LDXGOw)
Added unique business: Dunkin' (ID: Dh9eTTpWHCwmy7ESU-5azA)
Added unique business: Yard House (ID: Role_BlY9mBhq8Q4nTzdlw)
Added unique business: Bravo Kosher Pizza (ID: o59DSzc-X9aFvjuwCSVbPg)
Added unique business: Jollibee (ID: HNbdS73lZker0Xd1qu9mXQ)
Added unique business: Marcus Live! Bar & Grille (ID: t8GFj7gt3SeaiBbFEuwTzQ)
Added unique business: Around The Clock (ID: yuIUpbCBlZ653SMld_XKyA)
Added unique business: Five Guys (ID: SbauU8ft-0vNQ9DQ8q4GuQ)
Added unique business: Falafel (ID: uF_4r7k03coJUvO6aS9IJw)
Added unique business: A'jar Sweet Cream Factory (ID: Mv2V_q0GfL9xs-ljWysiEQ)
Added u